In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from skimage.morphology import skeletonize
from scipy.optimize import least_squares
from tqdm import tqdm

In [2]:
# Define Bézier Curve Function (Degree 4)
def bezier_curve(t, P0, P1, P2, P3, P4):
    return (
        (1 - t) ** 4 * P0 +
        4 * (1 - t) ** 3 * t * P1 +
        6 * (1 - t) ** 2 * t ** 2 * P2 +
        4 * (1 - t) * t ** 3 * P3 +
        t ** 4 * P4
    )

# Compute residuals between target slur points and Bézier curve
def bezier_residuals(params, t_vals, target_points, P0, P4):
    P1, P2, P3 = params.reshape(3, 2)  # Extract control points to optimize
    fitted_points = np.array([bezier_curve(t, P0, P1, P2, P3, P4) for t in t_vals])
    return (fitted_points - target_points).flatten()

# Convert control points to the correct format before saving
def format_point(point):
    return f"[{round(point[0], 3)}, {round(point[1], 3)}]"

In [3]:
def compute_bezier_control_points(image, bbox):
    # Convert to binary using Otsu's thresholding
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    row_pixel_count = np.sum(binary > 0, axis=1)  # Compute row-wise histogram
    max_pixel_count = np.max(row_pixel_count)  # Equal to image width in most cases
    rows_to_remove = set(np.where(row_pixel_count > 0.7 * max_pixel_count)[0])  # PARAMETER

    # Remove neighboring rows if above threshold
    for row in list(rows_to_remove):
        for neighbor in [row - 1, row + 1]:  # PARAMETER
            if 0 <= neighbor < binary.shape[0] and row_pixel_count[neighbor] > 0.2 * max_pixel_count:
                rows_to_remove.add(neighbor)  # Remove neighboring row if above the 20% threshold
    rows_to_remove = sorted(rows_to_remove)

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 17))  # PARAMETER
    stems = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=1)
    binary_no_stems = cv2.subtract(binary, stems)    # Remove note stems

    binary_cleaned = binary_no_stems.copy()
    binary_cleaned[rows_to_remove, :] = 0  # Set selected rows to black

    skeleton = skeletonize(binary_cleaned > 0)  # Apply skeletonization
    skeleton_cleaned = (skeleton * 255).astype(np.uint8)  # Convert back to uint8 format for visualization

    x_min, y_min, x_max, y_max = bbox
    w, h = x_max - x_min, y_max - y_min
    slur_crop = skeleton_cleaned[y_min - 20:y_max + 20, x_min - 20:x_max + 20]  # Crop the cleaned skeletonized image
    
    # Extract skeleton points inside the bounding box
    y_coords, x_coords = np.where(slur_crop > 0)
    points = np.column_stack((x_coords, y_coords))

    G = nx.Graph()    # Build graph and find slur segments
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            if np.linalg.norm(points[i] - points[j]) < 3:  # Connect nearby pixels
                G.add_edge(tuple(points[i]), tuple(points[j]))
    
    # Identify all disconnected components
    connected_components = [np.array(sorted(c, key=lambda p: p[0])) for c in nx.connected_components(G)]
    # print(len(connected_components))
    for i in range(len(connected_components)):  # Apply bounding box offsets
        connected_components[i][:, 0] += x_min - 20  # Shift x-coordinates
        connected_components[i][:, 1] += y_min - 20  # Shift y-coordinates
    
    # Find the initial longest component
    initial_longest = max(connected_components, key=len)
    # Remove components within the x-range of the initial slur
    x_min_slur, x_max_slur = np.min(initial_longest[:, 0]), np.max(initial_longest[:, 0])
    filtered_components = [
        c for c in connected_components if not (np.min(c[:, 0]) > x_min_slur and np.max(c[:, 0]) < x_max_slur)]

    G_loose = nx.Graph()  # Reconnect nearby components with looser definition
    for c in filtered_components:
        for i in range(len(c) - 1):  # Add filtered components to the new graph
            G_loose.add_edge(tuple(c[i]), tuple(c[i + 1]))
    
    # Connect components if their closest points are within distance < 15
    for i, comp1 in enumerate(filtered_components):
        for j, comp2 in enumerate(filtered_components):
            if i >= j:
                continue  # Avoid duplicate checks
            # Determine which component is on the left and right
            if np.min(comp1[:, 0]) < np.min(comp2[:, 0]):  
                left_component, right_component = comp1, comp2
            else:
                left_component, right_component = comp2, comp1
            rightmost_p1 = left_component[np.argmax(left_component[:, 0])]
            leftmost_p2 = right_component[np.argmin(right_component[:, 0])]
            min_distance = np.linalg.norm(rightmost_p1 - leftmost_p2)
            if min_distance < 15:  # PARAMETER
                G_loose.add_edge(tuple(rightmost_p1), tuple(leftmost_p2))  # Connect closest boundary points
    
    # Find the new longest component (final slur)
    final_components = [np.array(list(c)) for c in nx.connected_components(G_loose)]
    # print(len(final_components))

    longest_component = max(final_components, key=len)  # In case there are multiple components
    longest_component = longest_component[np.argsort(longest_component[:, 0])]  # Sort on x-values
    longest_component = longest_component.astype(np.float32)  # Normalize path for Bézier fitting
    x_vals, y_vals = longest_component[:, 0], longest_component[:, 1]
    
    # Initialize Bézier Control Points Based on Assumption
    P0 = np.array([x_vals[0], y_vals[0]])  # Start point
    P1 = np.array([np.mean(x_vals[:len(x_vals)//4]), np.max(y_vals)])  # try min and max
    P2 = np.array([np.mean(x_vals[len(x_vals)//4:len(x_vals)//2]), np.max(y_vals)])
    P3 = np.array([np.mean(x_vals[len(x_vals)//2:3*len(x_vals)//4]), np.max(y_vals)])
    P4 = np.array([x_vals[-1], y_vals[-1]])  # End point

    t_vals = np.linspace(0, 1, len(longest_component))
    target_path = np.column_stack((x_vals, y_vals))
    
    # Optimize control points using least squares fitting
    initial_guess = np.hstack([P1, P2, P3])
    res = least_squares(bezier_residuals, initial_guess, args=(t_vals, target_path, P0, P4))
    
    # Extract optimized control points
    P1_opt, P2_opt, P3_opt = res.x.reshape(3, 2)
    bezier_points = np.array([P0, P1_opt, P2_opt, P3_opt, P4])

    return bezier_points, x_vals, y_vals


In [4]:
df = pd.read_csv("control_points.csv")
df = df[df['Invalid Bezier Curve'] == 1].reset_index(drop=True)
os.makedirs("visual_invalid", exist_ok=True)
df[['P0', 'P1', 'P2', 'P3', 'P4']] = None

for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating Bézier Curves"):
    
    image_path = os.path.join("Use", row['Crop File Name'])
    if not os.path.exists(image_path):
        print(f"Skipping missing image: {image_path}")
        continue
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    bbox = eval(row['BoundingBox in Crop'])
    
    bezier_points, x_vals, y_vals = compute_bezier_control_points(image, bbox)
    
    # Save control points in DataFrame
    df.at[index, 'P0'] = format_point(bezier_points[0])
    df.at[index, 'P1'] = format_point(bezier_points[1])
    df.at[index, 'P2'] = format_point(bezier_points[2])
    df.at[index, 'P3'] = format_point(bezier_points[3])
    df.at[index, 'P4'] = format_point(bezier_points[4])

    # Generate fitted Bézier curve for visualization
    t_vals_fit = np.linspace(0, 1, 200)
    fitted_curve = np.array([bezier_curve(t, *bezier_points) for t in t_vals_fit])
    
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray', alpha=0.6)
    ax.set_title("Fitted Bézier Curve for Slur")
    ax.plot(x_vals, y_vals, 'bo', markersize=2, label="Skeletonized Slur Path")
    ax.plot(fitted_curve[:, 0], fitted_curve[:, 1], 'r-', linewidth=2, label="Fitted Bézier Curve")
    ax.scatter(bezier_points[:, 0], bezier_points[:, 1], color='yellow', label="Control Points", s=40)
    ax.legend()
    
    output_visual_path = os.path.join("visual_invalid", f"Bezier_{row['Crop File Name']}")
    plt.savefig(output_visual_path)
    plt.close()


# df.to_csv("control_points.csv", index=False)

Calculating Bézier Curves: 100%|███████████████████████████████████████████████████████| 40/40 [00:43<00:00,  1.09s/it]
